In [1]:
import numpy as np

Method 1. From https://arxiv.org/pdf/1803.06987 

In [2]:
def fftshift(x, axis=-1):
    """Cyclically shift along given axis (matches MATLAB fftshift)."""
    x = np.asarray(x)
    n = x.shape[axis]
    shift = n // 2
    return np.roll(x, shift=shift, axis=axis)

def symplectic_inner_product(u, v):
    n = len(u) // 2
    return (np.dot(u[:n], v[n:]) + np.dot(u[n:], v[:n])) % 2

def gf2_inverse(A):
    """Invert a binary matrix over GF(2)."""
    A = A.copy() % 2
    n = A.shape[0]
    I = np.eye(n, dtype=int)
    aug = np.hstack([A, I])
    for col in range(n):
        pivot = np.where(aug[col:, col] == 1)[0]
        if len(pivot) == 0:
            raise ValueError("Matrix not invertible in GF(2)")
        pivot = pivot[0] + col
        aug[[col, pivot]] = aug[[pivot, col]]
        for row in range(n):
            if row != col and aug[row, col]:
                aug[row] ^= aug[col]
    return aug[:, n:]

def bi2de(binary_array):
    return binary_array.dot(1 << np.arange(binary_array.shape[-1] - 1, -1, -1))

def de2bi(i, width):
    """Binary representation of integers as array of shape (len(i), width)"""
    i = np.atleast_1d(i)
    return ((i[:, None] & (1 << np.arange(width - 1, -1, -1))) > 0).astype(int)

def find_all_symp_mat(U, V, I, J, find_symp_mat):
    m = U.shape[0] // 2
    I = np.array(I).flatten()
    J = np.array(J).flatten()
    Ibar = np.setdiff1d(np.arange(m), I)
    Jbar = np.setdiff1d(np.arange(m), J)
    alpha = len(Ibar) + len(Jbar)
    tot = 2**(alpha * (alpha + 1) // 2)
    F_all = []

    F0 = find_symp_mat(np.vstack([U[I], U[m + J]]), V)
    A = U @ F0 % 2
    Ainv = gf2_inverse(A)
    IbJb = np.union1d(Ibar, Jbar)
    Basis = A[np.concatenate([IbJb, m + IbJb])]
    Subspace = (de2bi(np.arange(2**(2 * len(IbJb))), 2 * len(IbJb)) @ Basis) % 2

    # Identify fixed and free indices
    _, Basis_fixed_I, _ = np.intersect1d(IbJb, I, return_indices=True)
    _, Basis_fixed_J, _ = np.intersect1d(IbJb, J, return_indices=True)
    Basis_fixed = np.concatenate([Basis_fixed_I, len(IbJb) + Basis_fixed_J])
    Basis_free = np.setdiff1d(np.arange(2 * len(IbJb)), Basis_fixed)

    # Build Choices
    Choices = []
    for i in range(alpha):
        ind = Basis_free[i]
        h = np.zeros(len(Basis_fixed), dtype=int)
        if i < len(Ibar):
            h[np.where(Basis_fixed == len(IbJb) + ind)[0]] = 1
        else:
            h[np.where(Basis_fixed == ind - len(IbJb))[0]] = 1
        Innpdts = (Subspace @ fftshift(Basis[Basis_fixed], axis=1).T) % 2
        valid = np.where(bi2de(Innpdts) == bi2de(h))[0]
        Choices.append(Subspace[valid])

    # Enumerate all valid configurations
    for l in range(tot):
        Bl = A.copy()
        W = np.zeros((alpha, 2 * m), dtype=int)
        lbin = de2bi(l, alpha * (alpha + 1) // 2)[0]
        v1_ind = bi2de(lbin[:alpha])
        W[0] = Choices[0][v1_ind]
        offset = 0
        for i in range(1, alpha):
            chunk_len = alpha - i
            vi_bits = lbin[offset + alpha:offset + alpha + chunk_len]
            vi_ind = bi2de(vi_bits)
            offset += chunk_len
            Innprods = (Choices[i] @ fftshift(W[:i], axis=1).T) % 2
            h = np.zeros(i, dtype=int)
            if i >= len(Ibar):
                idx = np.where(Basis_free == Basis_free[i] - len(IbJb))[0]
                if len(idx) > 0:
                    h[idx[0]] = 1
            valid = np.where(bi2de(Innprods) == bi2de(h))[0]
            W[i] = Choices[i][vi_ind % len(valid)]
        Bl[np.concatenate([Ibar, m + Jbar])] = W
        F = (Ainv @ Bl) % 2
        F_all.append((F0 @ F) % 2)
    return F_all


def gf2_solve(A, b):
    """Solve Ax = b over GF(2) using Gaussian elimination."""
    A = A.copy() % 2
    b = b.copy() % 2
    m, n = A.shape
    aug = np.hstack([A, b.reshape(-1, 1)]).astype(np.uint8)

    row = 0
    for col in range(n):
        pivot_row = None
        for r in range(row, m):
            if aug[r, col] == 1:
                pivot_row = r
                break
        if pivot_row is None:
            continue
        aug[[row, pivot_row]] = aug[[pivot_row, row]]
        for r in range(m):
            if r != row and aug[r, col]:
                aug[r] ^= aug[row]
        row += 1

    x = np.zeros(n, dtype=np.uint8)
    for i in range(min(m, n)):
        pivot_cols = np.flatnonzero(aug[i, :n])
        if len(pivot_cols) == 1:
            x[pivot_cols[0]] = aug[i, -1]
    return x

def find_symp_mat(X, Y):
    """
    Find binary symplectic matrix F such that X @ F = Y over GF(2).
    Each row in X, Y must be length 2n (i.e., X.shape = (m, 2n)).
    """
    m, two_n = X.shape
    n = two_n // 2
    F = np.eye(2 * n, dtype=int)

    def Z_h(h):
        """Symplectic transvection matrix."""
        h = h % 2
        outer = np.outer(fftshift(h), h) % 2
        return (np.eye(2 * n, dtype=int) + outer) % 2

    def find_w(x, y, Ys):
        A = fftshift(np.vstack([x, y, Ys]), axis=1)
        b = np.concatenate([[1], [1], [symplectic_inner_product(y_j, y) for y_j in Ys]])
        return gf2_solve(A, b)

    for i in range(m):
        x_i = X[i]
        y_i = Y[i]
        x_it = (x_i @ F) % 2

        if np.array_equal(x_it, y_i):
            continue
        if symplectic_inner_product(x_it, y_i) == 1:
            h_i = (x_it + y_i) % 2
            F = (F @ Z_h(h_i)) % 2
        else:
            Ys_prev = Y[:i]
            w_i = find_w(x_it, y_i, Ys_prev)
            h_i1 = (w_i + y_i) % 2
            h_i2 = (x_it + w_i) % 2
            F = (F @ Z_h(h_i1)) % 2
            F = (F @ Z_h(h_i2)) % 2

    return F

Method 2

In [3]:
import numpy as np
from sympy import Matrix

def symplectic_form(n, d):
    J = np.block([[np.zeros((n, n), dtype=int), np.eye(n, dtype=int)],
                  [-np.eye(n, dtype=int), np.zeros((n, n), dtype=int)]]) % d
    return J

def gram_schmidt_symplectic(X, d):
    from sympy import Matrix

    X = Matrix(X.tolist())
    X = X.applyfunc(lambda x: x % d)

    n = X.rows
    m = X.cols

    basis = []
    for i in range(m):
        x = X.col(i)
        if x.norm() == 0:
            continue
        for b in basis:
            x -= x.dot(b) * b
            x = x.applyfunc(lambda xi: xi % d)
        if x.norm() == 0:
            continue
        basis.append(x)

    if not basis:
        return np.zeros((n, 0), dtype=int)
    B = Matrix.hstack(*basis)
    return np.array(B.tolist(), dtype=int) % d

def find_symplectic_map(X, Y, d):
    X = np.array(X, dtype=int) % d
    Y = np.array(Y, dtype=int) % d

    assert X.shape == Y.shape
    n2, k = X.shape
    assert n2 % 2 == 0
    n = n2 // 2

    J = symplectic_form(n, d)
    X_sym = gram_schmidt_symplectic(X, d)
    Y_sym = gram_schmidt_symplectic(Y, d)

    try:
        Xinv = Matrix(X_sym.tolist()).inv_mod(d)
    except:
        raise ValueError("X_sym is not invertible mod d")

    F = (Matrix(Y_sym.tolist()) * Xinv) % d
    F_np = np.array(F.tolist(), dtype=int)
    return F_np
    # if is_symplectic(F_np, d) and np.array_equal((F_np @ X) % d, Y % d):
    #     return F_np
    # else:
    #     raise ValueError("Failed to construct symplectic matrix")
d = 2
X = np.array([
    [1, 0, 1, 1],
    [0, 1, 0, 1],
    [0, 1, 0, 0],
    [1, 0, 0, 1]
], dtype=int)


Y = np.array([
    [1, 1, 0, 1],
    [0, 1, 1, 1],
    [0, 1, 0, 0],
    [1, 1, 1, 1]
], dtype=int)
F = find_symplectic_map(X, Y, d)
print("Found symplectic F:")
print(F)
print("Check (F @ X) % d == Y % d:")
print((F @ X) % d)
print("should equal:")
print(Y)

Found symplectic F:
[[0 1 0 1]
 [1 0 1 1]
 [0 0 1 0]
 [1 0 1 0]]
Check (F @ X) % d == Y % d:
[[1 1 0 0]
 [0 1 1 0]
 [0 1 0 0]
 [1 1 1 1]]
should equal:
[[1 1 0 1]
 [0 1 1 1]
 [0 1 0 0]
 [1 1 1 1]]


Test both and compare

In [4]:
def test_find_all_symp_mat():
    # Dimension 4 system (2 qudits, d=2)
    U = np.array([
        [1, 0, 1, 1],
        [0, 1, 0, 1],
        [0, 1, 0, 0],
        [1, 0, 0, 1],
        [0, 0, 1, 0],
        [0, 0, 0, 1],
        [1, 1, 0, 0],
        [0, 1, 1, 0]
    ], dtype=int)  # Shape (2m, 2n)

    # Subset of U transformed under a symplectic map
    V = np.array([
        [1, 1, 0, 1],
        [0, 1, 1, 1],
        [0, 1, 0, 0],
        [1, 1, 1, 1]
    ], dtype=int)

    I = [0, 1]
    J = [2, 3]

    F_all = find_all_symp_mat(U, V, I, J, find_symp_mat)

    print(f"Generated {len(F_all)} symplectic matrices:")
    for i, F in enumerate(F_all[:5]):  # Show only first 5 for brevity
        print(f"\nF[{i}]:\n{F}")
        # Check symplectic property if desired

test_find_all_symp_mat()

ValueError: could not broadcast input array from shape (4,) into shape (8,)

In [1]:

def gauss_mod_d(A, b, d):
    A = A.copy() % d
    b = b.copy() % d
    m, n = A.shape
    Ab = np.hstack([A, b.reshape(-1, 1)])

    row = 0
    pivot_cols = []
    for col in range(n):
        pivot_rows = np.where(Ab[row:, col] % d != 0)[0]
        if pivot_rows.size == 0:
            continue
        pivot = pivot_rows[0] + row
        Ab[[row, pivot]] = Ab[[pivot, row]]
        inv = modinv(Ab[row, col], d)
        Ab[row] = (Ab[row] * inv) % d
        for r in range(m):
            if r != row and Ab[r, col] % d != 0:
                Ab[r] = (Ab[r] - Ab[r, col] * Ab[row]) % d
        pivot_cols.append(col)
        row += 1
        if row == m:
            break

    # Extract particular solution (set free variables to 0)
    x = np.zeros(n, dtype=int)
    for i, col in enumerate(pivot_cols):
        x[col] = Ab[i, -1]
    return x

In [6]:
H = np.array([[0., 0., 0., 1.],
 [1., 0., 0., 0.],
 [1., 1., 0., 0.],
 [1., 1., 1., 1.]], dtype=int)

H_prime = np.array([[0., 0., 0., 1.],
 [1., 0., 1., 0.],
 [0., 1., 0., 0.],
 [0., 1., 1., 0.]], dtype=int)

F = find_symp_mat(H, H_prime)
print("F:")
print(F)
print("Check H @ F == H_prime mod 2:")
print((H @ F) % 2)
print("Should equal H_prime:")
print(H_prime)

F:
[[1 0 1 0]
 [1 1 1 0]
 [0 0 1 1]
 [0 0 0 1]]
Check H @ F == H_prime mod 2:
[[0 0 0 1]
 [1 0 1 0]
 [0 1 0 0]
 [0 1 1 0]]
Should equal H_prime:
[[0 0 0 1]
 [1 0 1 0]
 [0 1 0 0]
 [0 1 1 0]]
